# BUILD LLM Bootcamp Day 2

## Setup

### Install Libraries

In [1]:
!pip install peft transformers==4.34.0 tokenizers vllm==0.2.1.post1 bitsandbytes datasets absl-py==1.3.0

### Import Libraries

In [1]:
from datasets import Dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys
from ft_datasets.utils import Concatenator
import pandas as pd
pd.set_option('display.max_colwidth', None)

import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import os
from transformers import TrainerCallback
from contextlib import nullcontext
from transformers import default_data_collator, Trainer, TrainingArguments

### Load Base Model and Tokenizer

In [2]:
model_id="meta-llama/Llama-2-7b-chat-hf"
print('loading tokenizer')
tokenizer = LlamaTokenizer.from_pretrained(model_id)
print('loading model')
model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

loading tokenizer
loading model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Prepare Datasets
### Load Datasets

*NOTE: Fine-tuning using 100 records on this setup (compute and other resources) will take about ~5mins and on the entire dataset it will take over 1 hour.*

In [3]:
# do not run this cell in the live demo, it takes >20 minutes to run

# def prepare_stratified_dataset(path, seed = 42):
#     raw_df = pd.read_json(path, lines=True)
#     raw_df['id'] = raw_df.index
#     ds = Dataset.from_pandas(raw_df, split='train')
#     cl = ClassLabel(num_classes=4, names=["EN", "FR", "DE", "ES"])
#     new_features = ds.features.copy()
#     new_features['lang_label'] = cl
#     cl_d = {l : cl.str2int(l) for l in ["EN", "FR", "DE", "ES"]}
#     def convert_lang(sample):
#         sample['lang_label'] = cl_d[sample['language']]
#         return sample
#     ds = ds.map(convert_lang, features=new_features)
#     ds_split = ds.train_test_split(test_size=0.15, stratify_by_column='lang_label', seed=42)
#     test_ds_split = ds_split['test'].train_test_split(test_size=2/3, stratify_by_column='lang_label', seed=42)
#     return ds_split['train'].to_pandas(), test_ds_split['train'].to_pandas(), test_ds_split['test'].to_pandas()

# train_df, eval_df, test_df = prepare_stratified_dataset(
#     '../transcripts.json'
# )

Map:   0%|          | 0/1368 [00:00<?, ? examples/s]

In [4]:
df = pd.read_json("../transcripts.json", lines=True)
print(f"Total records: {df.shape[0]}")
train_df = df.head(100)
print(f"Train        : {train_df.shape[0]}")
eval_df = df[200:300]
print(f"Eval         : {eval_df.shape[0]}")
test_df = df.tail(100)
print(f"Test         : {test_df.shape[0]}")

Total records: 1368
Train        : 100
Eval         : 100
Test         : 100


In [70]:
train_df.head()

,instruction,input,output,language,id,lang_label
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Greetings! I'm Frosty, your voice bot assistant for the holiday season. What's your name?\ncaller: Hey Frosty, my name is James.\nfrosty: Hi James! Where are you calling from?\ncaller: I'm calling from Sydney.\nfrosty: Nice to meet you, James from Sydney! Let's get started with your holiday wish list. What toy would you like to choose?\ncaller: Hmm... I'm not sure yet. What do you recommend?\nfrosty: I'm happy to help you explore our catalog! Does your child like plush toys?\ncaller: Yes, my little one definitely loves plush toys.\nfrosty: We have a sweet 2023 Holiday Fox 12-Inch Plush, or how about the Sesame Street Monster Meditation Elmo? Both are soft and cuddly.\ncaller: Oh, Elmo for sure!\nfrosty: The Sesame Street Monster Meditation Elmo is an excellent choice. Tell me, what do you enjoy the most about this time of the year?\ncaller: I love seeing all the decorations and spending time with my family.\nfrosty: That's wonderful! Do you have any favorite holiday memories?\ncaller: One year, we had a fantastic snowball fight in the park with our friends. It was so much fun!\nfrosty: That sounds like a terrific memory! So, are we adding Sesame Street Monster Meditation Elmo to your wish list?\ncaller: Yes! And you know what? Add the fox plushie as well.\nfrosty: You got it, adding the 2023 Holiday Fox 12-Inch Plush and Sesame Street Monster Meditation Elmo to your wish list. Have a great holiday season, James!\ncaller: Thanks, Frosty! You too!""","{""toy_list"": [""2023 Holiday Fox 12-Inch Plush"", ""Sesame Street Monster Meditation Elmo""], ""location"": ""Sydney""}",EN,245,0
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi there!\nfrosty: Hello! Welcome to our holiday hotline, I'm Frosty. How can I assist you today?\ncaller: Hey, Frosty. My name is Mike, and I live in Los Angeles.\nfrosty: Hi, Mike from Los Angeles! What can I help you with?\ncaller: I want to submit my holiday wish list, but I'm not sure what toys my son would like.\nfrosty: No problem! I'd be happy to help you explore our catalog. What are some of his interests?\ncaller: He loves playing with new gadgets and is a fan of Star Wars.\nfrosty: How about Star Wars LOLA animatronic droid? It's a fun gadget for any Star Wars fan.\ncaller: That sounds interesting. What else do you have?\nfrosty: Another option could be a Dog-E, the robot dog. It's a popular gadget for kids this holiday season.\ncaller: Let's go with the lola droid. But I also need a second toy.\nfrosty: Alright! Can you tell me more about what he enjoys during this time of the year?\ncaller: He loves playing outdoors and making snow forts with his friends.\nfrosty: In that case, he might be interested in the Tiny Land kids fort building kit. It allows him to create forts both indoors and outdoors, perfect for all seasons.\ncaller: Great, add both the lola droid and fort building kit to our wish list, please.\nfrosty: Excellent choices, Mike! I'll add the Star Wars LOLA animatronic droid and the Tiny Land kids fort building kit to your wish list. Have a fantastic holiday season!\n""","{""toy_list"": [""Star Wars LOLA animatronic droid"", ""Tiny Land kids fort building kit""], ""location"": ""Los Angeles""}",EN,968,0
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""\ncaller: Hello, I'm trying to submit a holiday wish list for my child but I'm not sure where to start.\nfrosty: Hello and welcome! I'd be happy to help. May I have your name please, and where are you calling from?\ncaller: My name is Laura, and I'm from Birmingham, Engla

In [4]:
datasets = {
    'train': Dataset.from_pandas(train_df),
    'eval': Dataset.from_pandas(eval_df),
    'test': Dataset.from_pandas(test_df)
}

### Apply Prompt to Datasets

In [5]:
train_prompt = f"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
{{instruction}}
### Input:
{{input_}}
### Output:
{{output}}
{{eos_token}}
"""

eval_prompt = f"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
{{instruction}}
### Input:
{{input_}}
### Output:
{{eos_token}}
"""

In [6]:
def apply_train_template(sample):
    return {
        "text": train_prompt.format(
            instruction=sample["instruction"],
            input_=sample["input"].replace('\\n', '\n'),
            output=sample["output"],
            eos_token=tokenizer.eos_token,
        )
    }

def apply_eval_template(sample):
    return {
        "text": eval_prompt.format(
            instruction=sample["instruction"],
            input_=sample["input"].replace('\\n', '\n'),
            eos_token=tokenizer.eos_token,
        )
    }



#applying template

datasets['train'] = datasets['train'].map(apply_train_template, remove_columns=list(datasets['train'].features))
for k in ['eval', 'test']:
    datasets[k] = datasets[k].map(apply_eval_template, remove_columns=list(datasets[k].features))


Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

### Tokenize Datasets

In [7]:
for k, v in datasets.items():
    datasets[k] = v.map(
        lambda sample: tokenizer(sample["text"]),
        batched=True,
        remove_columns=list(v.features),
    ).map(Concatenator(), batched=True)

Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

## Fine-tune Llama 2 Model


Fine-tuning is one form of model training. We start training from a pre-trained model and adjust a set of model parameters to better solve for a concrete task based on task specific data. Today we are going to fine-tune 7B Llama 2 model using LoRA (Low-Rank Adaptation)--which is a parameter efficient way of fine-tuning LLM. 

Instead of adjusting all the ~7B parameters, LoRA allows us to adjust only a percent of model weights--which can save compute and memory resources dramatically. For this lab, we will fine-tune our model using LoRA on a single A10 GPU. This will demostrate how good the inference can be on fine-tuned models even with limited compute.

### Configuration

We're passing `train_dataset` and `eval_dataset` that are used to generate loss calculation during fine-tuning process and we've set `output_weights_dir` as the directory where the fine-tuned weights will be stored after fine-tuning job completes.

We're also setting `max_steps=1` with 4bit quantization. Quantization is required to train on single A10 GPU. To achieve good performance for the task, you will need at least 1 `num_epochs`, feel free to explore this on your own.

In [8]:
#setting the model into training mode
model.train()

#setting up training
def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_kbit_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [9]:
# rm -r output_weights_dir # deletes prior fine tuning weights
!mkdir output_weights_dir

In [11]:
output_dir = "output_weights_dir"
enable_profiler = False

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

### Finetuning

In [12]:
profiler = nullcontext() 

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=datasets['train'],
        eval_dataset=datasets['eval'],
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()
    
model.save_pretrained(output_dir)

Detected kernel version 5.4.181, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/envs/rapids/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/envs/rapids/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,1.377400
20,1.095900
30,0.994500
40,0.950900
50,0.892900
60,0.841000
70,0.800400
80,0.791100


## Return to Local Jupyter Lab

we have finished fine tuning and will now return to our local jupyter lab environment. 